In [ ]:
import numpy as np
import torch
import math
from smplpytorch.pytorch.smpl_layer import SMPL_Layer
from display_utils import display_model

# ========== Rotation Utilities ==========
def axis_angle_to_matrix(axis_angle):
    """Convert axis-angle to rotation matrix (Rodrigues formula)"""
    theta = torch.norm(axis_angle, dim=-1, keepdim=True) + 1e-8
    axis = axis_angle / theta
    x, y, z = axis[..., 0], axis[..., 1], axis[..., 2]

    cos = torch.cos(theta)
    sin = torch.sin(theta)
    one_minus_cos = 1 - cos

    rot = torch.zeros(axis.shape[:-1] + (3, 3), dtype=axis.dtype, device=axis.device)
    rot[..., 0, 0] = cos + x * x * one_minus_cos
    rot[..., 0, 1] = x * y * one_minus_cos - z * sin
    rot[..., 0, 2] = x * z * one_minus_cos + y * sin

    rot[..., 1, 0] = y * x * one_minus_cos + z * sin
    rot[..., 1, 1] = cos + y * y * one_minus_cos
    rot[..., 1, 2] = y * z * one_minus_cos - x * sin

    rot[..., 2, 0] = z * x * one_minus_cos - y * sin
    rot[..., 2, 1] = z * y * one_minus_cos + x * sin
    rot[..., 2, 2] = cos + z * z * one_minus_cos
    return rot

def matrix_to_axis_angle(R):
    """Convert rotation matrix to axis-angle (inverse Rodrigues)"""
    cos_theta = (torch.diagonal(R, dim1=-2, dim2=-1).sum(-1) - 1) / 2
    theta = torch.acos(torch.clamp(cos_theta, -1.0, 1.0))

    axis = torch.stack([
        R[..., 2, 1] - R[..., 1, 2],
        R[..., 0, 2] - R[..., 2, 0],
        R[..., 1, 0] - R[..., 0, 1]
    ], dim=-1) / (2 * torch.sin(theta).unsqueeze(-1) + 1e-8)

    axis_angle = axis * theta.unsqueeze(-1)
    return axis_angle

# ========== Setup ==========
cuda = torch.cuda.is_available()

smpl_layer = SMPL_Layer(center_idx=0, gender='neutral', model_root='smplpytorch/native/models')
if cuda:
    smpl_layer = smpl_layer.cuda()

motion = np.load(r"C:\Users\lalas\Downloads\animation\comp_robot\zhangyuhong1\data\Motion-X++\v7\motion\motion_generation\smplx_322\animation\Ways_to_Jump_+_Sit_+_Fall_Reading_to_Children_clip1.npy")
motion = torch.tensor(motion).float()

motion_parms = {
    'root_orient': motion[:, :3],
    'pose_body': motion[:, 3:3+63],
    'pose_hand': motion[:, 66:66+90],
    'pose_jaw': motion[:, 66+90:66+93],
    'face_expr': motion[:, 159:159+50],
    'face_shape': motion[:, 209:209+100],
    'trans': motion[:, 309:309+3],
    'betas': motion[:, 312:],
}

# Append 6 zeros to pose_body for SMPL compatibility
zeros = torch.zeros(motion_parms['pose_body'].size(0), 6)
motion_parms['pose_body'] = torch.cat((motion_parms['pose_body'], zeros), dim=1)

if cuda:
    for k in motion_parms:
        motion_parms[k] = motion_parms[k].cuda()

# Rotation matrix for -90° about X
rot_x = torch.tensor([[-math.pi / 2, 0.0, 0.0]], dtype=torch.float32)
if cuda:
    rot_x = rot_x.cuda()
rot_x_mat = axis_angle_to_matrix(rot_x)  # [1, 3, 3]

# ========== Main Loop ==========
for frame_index in range(motion_parms['root_orient'].shape[0]):
    root_orient = motion_parms['root_orient'][frame_index].unsqueeze(0)  # [1, 3]
    body_pose = motion_parms['pose_body'][frame_index].unsqueeze(0)     # [1, 69]
    translation = motion_parms['trans'][frame_index].unsqueeze(0)
    shape_params = motion_parms['betas'][frame_index].unsqueeze(0)

    # Convert root_orient to matrix
    root_mat = axis_angle_to_matrix(root_orient)  # [1, 3, 3]
    corrected_root_mat = torch.matmul(rot_x_mat, root_mat)  # Apply -90 deg rotation
    corrected_root_orient = matrix_to_axis_angle(corrected_root_mat)  # [1, 3]

    pose_params = torch.cat([corrected_root_orient, body_pose], dim=1)

    # Run SMPL
    verts, Jtr = smpl_layer(pose_params, th_betas=shape_params)
    verts += translation.unsqueeze(1)
    Jtr += translation.unsqueeze(1)

    # CPU for visualization
    verts_cpu = verts.cpu().detach()
    Jtr_cpu = Jtr.cpu().detach()

    save_path = fr"C:\Users\lalas\Desktop\1\image_{frame_index:04d}.png"
    display_model(
        {'verts': verts_cpu, 'joints': Jtr_cpu},
        model_faces=smpl_layer.th_faces,
        with_joints=True,
        kintree_table=smpl_layer.kintree_table,
        savepath=save_path,
        show=False
    )

    # Uncomment if only testing one frame
    # break

print("Processing complete.")
